In [22]:
import os
import re
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageDraw  
from PIL import ImageFont
from moviepy.editor import *
import moviepy.audio.fx.all as afx

class psychoCards:
    def __init__(self,xlsFn,title):
        self.dir='/home/jack/data/大智小超/心理'
#         self.dir='i:/大智小超/心理'
        self.xlsFn=xlsFn
        self.title=title
        self.bgm=AudioFileClip(os.path.join(self.dir,self.title,self.title+'.mp3')).set_start(1)
    
    def makeList(self):
        fn=os.path.join(self.dir,self.xlsFn)
        ctns=pd.read_excel(fn)
#         print(ctns['标题二'],ctns['内容'],ctns['详细'])
        lst=np.array(ctns.loc[ctns['标题二']==self.title]['标题二']).tolist()
        lst.extend(ctns.loc[ctns['标题二']==self.title]['内容'])
#         lst.extend(ctns.loc[ctns['标题二']==self.title]['详细'])
#         print(lst)

        return lst
    
    def makePics(self,bg):
        txts=self.makeList()
        title=txts[0]
        txt=txts[1].split('\n')
        img=Image.open(os.path.join(self.dir,bg))
        
        font_title = ImageFont.truetype('/home/jack/files/FONTS/msyh.ttf',30)
        font_txt = ImageFont.truetype('/home/jack/files/FONTS/msyh.ttf',20)
#         font_title = ImageFont.truetype('C:\Windows\Fonts\msyh.ttc',30)
#         font_txt = ImageFont.truetype('C:\Windows\Fonts\msyh.ttc',20)
        draw = ImageDraw.Draw(img)            
        draw.text((40,int(img.size[1]*0.2)), title, fill = '#6AB34A',font=font_title)
        
        out=os.path.join(self.dir,self.title,'_img00.jpg')
        img.save(out)
        
        n=0
        for t in txt:
            img=Image.open(os.path.join(self.dir,self.title,'_img'+str(n).zfill(2)+'.jpg'))
            draw = ImageDraw.Draw(img)
            draw.text((40,int(img.size[1]*0.3+n*32)), t, fill = '#6AB34A',font=font_txt)
            img.save(os.path.join(self.dir,self.title,'_img'+str(n+1).zfill(2)+'.jpg'))
            n+=1
        
        
    def makeMovie(self):
        self.makePics()
        lst=self.makeList()
        drtns=[]        
        s=0
        f=0
        total_c=len(lst[1])
        total_time=self.bgm.duration
        spd=total_time/total_c
        
        txt=[lst[0]]
        txt.extend(lst[1].split('\n'))
        print(txt)
        
        drtns=[]
        for t in txt:
            drtns.append(len(t)*spd)        
        drtns[0]=1
#         print(drtns)
        print('正在生成影片……')
        pics=self.re_name()
        clips=[]
        k=0
        for pic in pics:
            if pic.lower()[-3:]=='jpg':
                print(pic,txt[k],f,drtns[k])
                _img=ImageClip(os.path.join(self.dir,self.title,pic)).set_fps(25).set_start(f).set_duration(drtns[k]).resize((480,640))
                f=f+drtns[k]
                k+=1         
                
                clips.append(_img)
        finalclip = CompositeVideoClip(clips)
        
        
        
        # 添加语音
        empty_1s=AudioFileClip(os.path.join(self.dir,'empty1s.mp3')).set_start(0)
        
#         print(BGM.duration)
        final_audio = CompositeAudioClip([empty_1s,self.bgm])
        mix=finalclip.set_audio(final_audio)
        
        out=os.path.join(self.dir,self.title,self.title+'.mp4')
        mix.write_videofile(out)
        
        print('完成影片')
        
    def makeMovie_ppt(self):
        lst=self.makeList()
        drtns=[]        
        s=0
        f=0
        total_c=len(lst[1])
        total_time=self.bgm.duration
        spd=total_time/total_c
        
        txt=[lst[0]]
        txt.extend(lst[1].split('\n'))
        print(txt)
        
        drtns=[]
        for t in txt:
            drtns.append(len(t)*spd)        
        drtns[0]=1
        print(drtns)
        print('正在生成影片……')
        pics=self.re_name()
        clips=[]
        k=0
        for pic in pics:
            if pic.lower()[-3:]=='jpg':
                print(pic,txt[k],f,drtns[k])
                _img=ImageClip(os.path.join(self.dir,self.title,pic)).set_fps(25).set_start(f).set_duration(drtns[k]).resize((480,640))
                f=f+drtns[k]
                k+=1         
                
                clips.append(_img)
        finalclip = CompositeVideoClip(clips)
        
        
        
        # 添加语音
        empty_1s=AudioFileClip(os.path.join(self.dir,'empty1s.mp3')).set_start(0)
        
#         print(BGM.duration)
        final_audio = CompositeAudioClip([empty_1s,self.bgm])
        mix=finalclip.set_audio(final_audio)
        
        out=os.path.join(self.dir,self.title,self.title+'.mp4')
        mix.write_videofile(out)
        
        print('完成影片')
    
    def re_name(self):
        print('重命名文件以排序……')
        old=os.path.join(self.dir,self.title)
        fn_list=os.listdir(old)
        ptn=r'幻灯片.*jpg'
        n=len(fn_list)
        m=0
        for fn in fn_list:
            q=re.match(ptn,fn.lower())
            if q and len(fn)<9:
                new=os.path.join(self.dir,self.title,fn[0:3]+'0'+fn[3:])
                os.rename(os.path.join(old,fn),new)
                m+=1
                
            if m>0:
                p='该文件夹内共有{0}个文件，重命名{1}个。'.format(n,m)
            else:
                p='该文件夹内共有{0}个文件，没有符合重命名条件的文件。'.format(n)
        print(p)
        new_list=os.listdir(old)
        new_list.sort()
#         print(new_list)
        return new_list
        
if __name__=='__main__':
    mycard=psychoCards('正面管教.xlsx','我的孩子不惩罚真的管不住怎么办')
#     mycard.makeMovie_ppt()
    mycard.makePics('/home/jack/data/大智小超/心理/bg01.jpg')
    